In [ ]:
!pip install --upgrade plotly

     |████████████████████████████████| 26.5 MB 1.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
plotly.__version__

'5.5.0'

In [ ]:
INPUT_PATH = '../../data/3_processed/forms'
OUTPUT_PATH = '../../data/4_content'

## Reading

In [ ]:
enem = pd.read_csv(os.path.join(INPUT_PATH, 'enem.csv')).convert_dtypes()
enem.head()

,id,unidade,curso,ano,modalidade,chamada,nota_linguagens,nota_humanas,nota_natureza,nota_matematica,nota_redacao,nota_final,nota_redacao_c1,nota_redacao_c2,nota_redacao_c3,nota_redacao_c4,nota_redacao_c5,redacao
0,12,EACH,Gestão de Políticas Públicas,2021,Escola Pública (EP),Primeira,750.0,790.0,700.0,670.0,960,713.00,200,200,200,160,200,<NA>
1,0,EACH,Sistemas de Informação,2021,"Pretos, Pardos e Indígenas (PPI)",Primeira,659.3,666.7,627.1,703.6,880,707.34,160,180,160,180,200,<NA>
2,24,ECA,Com. Social: Editoração,2020,Ampla Concorrência (AC),Primeira,669.6,688.0,640.9,838.7,980,793.00,180,200,200,200,200,https://drive.google.com/open?id=1OWcisZ_cYUSV...
3,24,ECA,Com. Social: Editoração,2021,Escola Pública (EP),Primeira,626.1,717.5,590.9,636.6,980,761.63,0,0,0,0,0,<NA>
4,174,ECA,Relações Públicas,2021,"Pretos, Pardos e Indígenas (PPI)",Primeira,697.0,659.3,583.7,657.2,920,757.32,160,200,200,160,200,<NA>


In [ ]:
fuvest = pd.read_csv(os.path.join(INPUT_PATH, 'fuvest.csv')).convert_dtypes()
fuvest.head()

,id,unidade,curso,ano,modalidade,chamada,nota_1,nota_2_1,nota_2_2,nota_final,nota_redacao,classificacao_carreira,classificacao_curso,redacao
0,4,EACH,Biotecnologia,<NA>,Ampla Concorrência (AC),Primeira,69,53.00,60.00,632.22,31.50,25,25,<NA>
1,4,EACH,Biotecnologia,<NA>,Ampla Concorrência (AC),Primeira,64,60.50,53.33,616.48,36.50,28,28,<NA>
2,4,EACH,Biotecnologia,<NA>,Ampla Concorrência (AC),Primeira,69,55.17,62.50,647.78,30.67,2222,2222,https://drive.google.com/open?id=1xdkKGfCqlV1f...
3,4,EACH,Biotecnologia,2021,Ampla Concorrência (AC),Primeira,62,57.00,65.83,639.07,36.50,24,24,https://drive.google.com/open?id=1BTQ5QcG0wVyg...
4,4,EACH,Biotecnologia,2021,Ampla Concorrência (AC),Primeira,78,77.00,83.33,823.33,46.50,1,1,https://drive.google.com/open?id=13HroIuXzfwRp...


## Pre-processing

In [ ]:
fuvest.ano = fuvest.ano.fillna(2021)
enem.ano = enem.ano.fillna(2021)

## Analysing

### Support functions

In [ ]:
def table_from_df(df, cols=None):
  if cols is None:
    cols = df.columns
    
  data_list = [df[col] for col in cols]
  fig = go.Figure(data=go.Table(
      header=dict(values=cols),
      cells=dict(values=data_list)
  ))
  return fig

In [ ]:
def plotly_fig_to_html(fig, path):
  fig.write_html(path, include_plotlyjs='cdn', full_html=False)

### Fuvest

In [ ]:
for index, row in fuvest[['id', 'ano']].drop_duplicates().iterrows():
  os.makedirs(os.path.join(OUTPUT_PATH, str(row.id), 'fuvest', str(row.ano)))

In [ ]:
fuvest_grouped = fuvest.groupby(['id', 'ano', 'modalidade'])
fuvest_grouped.groups

{(0, 2019, 'Pretos, Pardos e Indígenas (PPI)'): [42], (0, 2021, 'Ampla Concorrência (AC)'): [39, 40, 43, 45, 47, 49, 50, 53, 54, 55, 56, 57, 59, 61, 63, 64], (0, 2021, 'Escola Pública (EP)'): [41, 44, 46, 48, 51, 52, 58, 60, 62], (2, 2021, 'Escola Pública (EP)'): [24], (4, 2021, 'Ampla Concorrência (AC)'): [0, 1, 2, 3, 4, 5, 6, 8], (4, 2021, 'Escola Pública (EP)'): [7], (6, 2021, 'Ampla Concorrência (AC)'): [37, 38], (6, 2021, 'Escola Pública (EP)'): [36], (7, 2021, 'Ampla Concorrência (AC)'): [65, 67, 71, 72, 74], (7, 2021, 'Escola Pública (EP)'): [66, 68, 69, 70], (7, 2021, 'Pretos, Pardos e Indígenas (PPI)'): [73], (10, 2021, 'Ampla Concorrência (AC)'): [9, 10, 11, 17], (10, 2021, 'Escola Pública (EP)'): [12, 13, 14, 15, 16], (12, 2020, 'Ampla Concorrência (AC)'): [23], (12, 2021, 'Ampla Concorrência (AC)'): [18, 20, 21, 22], (12, 2021, 'Pretos, Pardos e Indígenas (PPI)'): [19], (14, 2020, 'Ampla Concorrência (AC)'): [30], (14, 2021, 'Ampla Concorrência (AC)'): [27, 31, 33, 34, 35],

In [ ]:
fuvest_metrics_df = fuvest_grouped.agg(
  min=('nota_final', 'min'),
  max=('nota_final', 'max'), 
  mean=('nota_final', 'mean'),
)
fuvest_metrics_df

min     max        mean
id  ano  modalidade                                                  
0   2019 Pretos, Pardos e Indígenas (PPI)  474.63  474.63  474.630000
    2021 Ampla Concorrência (AC)           577.96  729.81  649.513750
         Escola Pública (EP)               479.44  579.26  527.895556
2   2021 Escola Pública (EP)               352.78  352.78  352.780000
4   2021 Ampla Concorrência (AC)           616.48  823.33  674.373750
...                                           ...     ...         ...
171 2021 Pretos, Pardos e Indígenas (PPI)  378.33  486.11  432.220000
173 2021 Ampla Concorrência (AC)           500.37  776.67  580.865000
         Escola Pública (EP)               420.37  542.22  486.887500
174 2021 Ampla Concorrência (AC)           764.26  764.26  764.260000
         Escola Pública (EP)               643.70  643.70  643.700000

[243 rows x 3 columns]

#### Metrics plot

In [ ]:
example_metrics = fuvest_metrics_df.loc[0].reset_index()
example_metrics

,ano,modalidade,min,max,mean
0,2019,"Pretos, Pardos e Indígenas (PPI)",474.63,474.63,474.630000
1,2021,Ampla Concorrência (AC),577.96,729.81,649.513750
2,2021,Escola Pública (EP),479.44,579.26,527.895556


In [ ]:
molten_example_metrics = pd.melt(example_metrics,
                                 id_vars=['ano', 'modalidade'],
                                 value_vars=['min', 'max', 'mean'],
                                 var_name='metric',
                                 value_name='nota')
molten_example_metrics

,ano,modalidade,metric,nota
0,2019,"Pretos, Pardos e Indígenas (PPI)",min,474.630000
1,2021,Ampla Concorrência (AC),min,577.960000
2,2021,Escola Pública (EP),min,479.440000
3,2019,"Pretos, Pardos e Indígenas (PPI)",max,474.630000
4,2021,Ampla Concorrência (AC),max,729.810000
5,2021,Escola Pública (EP),max,579.260000
6,2019,"Pretos, Pardos e Indígenas (PPI)",mean,474.630000
7,2021,Ampla Concorrência (AC),mean,649.513750
8,2021,Escola Pública (EP),mean,527.895556


In [ ]:
px.line(
  molten_example_metrics,
  x='ano', y='nota',
  color='metric',
  facet_row='modalidade',
  markers=True,
)

In [ ]:
figs = []
for id, df in fuvest_metrics_df.groupby('id'):
  molten_df = pd.melt(df.reset_index(),
                      id_vars=['ano', 'modalidade'],
                      value_vars=['min', 'max', 'mean'],
                      var_name='metric',
                      value_name='nota')
  fig = px.line(
    molten_df,
    x='ano', y='nota',
    color='metric',
    facet_row='modalidade',
    markers=True, # for line chart
    # barmode='group', # for bar chart
  )
  figs.append(fig)

print(len(figs))
figs[0].show()

112


In [ ]:
figs[102].show()

In [ ]:
def plot_fuvest_metric_evolution(fuvest_metrics_df):
  for id, df in fuvest_metrics_df.groupby('id'):
    molten_df = pd.melt(df.reset_index(),
                        id_vars=['ano', 'modalidade'],
                        value_vars=['min', 'max', 'mean'],
                        var_name='metric',
                        value_name='nota')
    fig = px.line(
      molten_df,
      x='ano', y='nota',
      color='metric',
      facet_row='modalidade',
      markers=True, # for line chart
      # barmode='group', # for bar chart
    )

    plotly_fig_to_html(
      fig,
      os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                   'metric_evolution.html'))

In [ ]:
plot_fuvest_metric_evolution(fuvest_metrics_df)

#### Metrics table for the latest year

In [ ]:
metric_tables = []
latest_year = fuvest_metrics_df.index.get_level_values('ano').max()
for id, df in(
      fuvest_metrics_df.loc[
        fuvest_metrics_df.index.get_level_values('ano') == latest_year
      ].reset_index().groupby('id')
    ):
  metric_tables.append(table_from_df(df, ['modalidade', 'min', 'mean', 'max']))

print(len(metric_tables))
metric_tables[0].show()

112


In [ ]:
def plot_fuvest_latest_metrics_table(fuvest_metrics_df):
  latest_year = fuvest_metrics_df.index.get_level_values('ano').max()
  for id, df in(
        fuvest_metrics_df.loc[
          fuvest_metrics_df.index.get_level_values('ano') == latest_year
        ].reset_index().groupby('id')
      ):
    plotly_fig_to_html(
      table_from_df(df, ['modalidade', 'min', 'mean', 'max']),
      os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                   'latest_metrics.html'))

In [ ]:
plot_fuvest_latest_metrics_table(fuvest_metrics_df)

#### Data tables

In [ ]:
data_tables = []
for id, df in fuvest.groupby('id'):
  df = df.reset_index()
  for ano in df['ano'].unique():
    for modalidade in df['modalidade'].unique():
      sliced_df = df[(df['ano'] == ano)
                     & (df['modalidade'] == modalidade)]
      if not sliced_df.empty:
        sliced_df = sliced_df.sort_values('nota_final', ascending=False)
        data_tables.append(table_from_df(sliced_df, sliced_df.columns[6:-1]))

print(len(data_tables))
data_tables[0].show()

243


In [ ]:
data_tables[4].show()

In [ ]:
def plot_fuvest_data_tables(fuvest_df, init_data_col=6, final_data_col=-1):
  for id, df in fuvest.groupby('id'):
    df = df.reset_index()
    for ano in df['ano'].unique():
      for modalidade in df['modalidade'].unique():
        sliced_df = df[(df['ano'] == ano)
                      & (df['modalidade'] == modalidade)]
        if not sliced_df.empty:
          sliced_df = sliced_df.sort_values('nota_final', ascending=False)
          plotly_fig_to_html(
            table_from_df(sliced_df, 
                          sliced_df.columns[init_data_col:final_data_col]),
            os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                         str(ano), f'{modalidade}.html')  
          )

In [ ]:
plot_fuvest_data_tables(fuvest)